In [39]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import os
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import requests as req
from bs4 import BeautifulSoup
import re
import datetime
from datetime import date
from os import listdir
import glob

In [40]:
lista_keywords = ['Negro de mierda','Mantero','Extranjeros ilegales','panchito','panchitos','maricón','Sudaca','espaldas mojadas','indio','trasvesti','inmigrantes ilegales','Moro','Moros','Travelo','Paki','Bujarra','sidoso','negrata','Bollera','mariposon','desviado','mariquita','sarasa','machorra','tortillera','comebollos','machurrona','Africano','Latino','virus chino','Homosexual','machupichu','Tiraflechas','afeminado','negrata','indígenas','Sin papeles']

In [41]:
def url_extractor(lista):
    options = webdriver.ChromeOptions()
    options.add_argument('disable-infobars')
    count = 1
    for keyword in lista:
        #Apertura de página con selenium
        listing_links = []
        
        link = 'https://www.abc.es/hemeroteca/'+ keyword
        print(keyword)
        
        driver = webdriver.Chrome(executable_path="../drivers/chromedriver")
        driver.get(link)
        time.sleep(1)
    
        #Extracción de links con selenium.
        try:
            for e in range(1, 1000):  

                link = 'https://www.abc.es/hemeroteca/'+ keyword + '/pagina-' + str(e)
                print(link)
                driver = webdriver.Chrome(executable_path="../drivers/chromedriver")
                driver.get(link)
                time.sleep(1)

                resultados = driver.find_elements_by_xpath('//*[@id="ctl00_CtrlPestanyas_basicaPN"]/fieldset')
                for e in resultados:
                    print(e.text)
                    texto= e.text.split('\n')[1]
                    print(texto)
                actions = ActionChains(driver) 
                #Scroll para que aparezca el link a la siguiente página
                for _ in range(3):
                    actions.send_keys(Keys.PAGE_DOWN).perform()
                time.sleep(1)

                links = driver.find_elements_by_css_selector('.titulo')
                for link in links:
                    listing_links.append(str(link.get_attribute('href')))
                driver.close()
                          
                listing_links = [x for x in listing_links if x != 'None']

                if texto == 'La búsqueda no obtuvo ningún resultado.':
                    break

        except:
            continue
            
        #creación de dataframe con los datos generados
        listing_links = list(dict.fromkeys(listing_links))
        df = pd.DataFrame(listing_links, columns=['link'])
        df['keyword'] = keyword
        #guardado de archivos, con el nombre de su keyword
        nombre_archivo = keyword + '.csv'
        path = './news_urls/diario_ABC_urls'
        archivo_de_salida = os.path.join(path,nombre_archivo)

        df.to_csv(archivo_de_salida, index=False)



In [42]:
def concatenator():
    path = './news_urls/diario_ABC_urls'
    e = glob.glob(path + "/*.csv")
    dfs = []
    for i in e:
        dfs.append(pd.read_csv(i))
    df_abc = pd.concat(dfs, axis=0)
    return df_abc

In [43]:
def scrap_news(df_abc):
    options = webdriver.ChromeOptions()
    options.add_argument('disable-infobars')
    links = df_abc['link'].tolist()
    df_list = []
    for e in links:
        try:
            
            link = e
            driver = webdriver.Chrome(executable_path="../DFGM/drivers/chromedriver")
            driver.get(link)
            time.sleep(1)
            resultados = driver.find_elements_by_css_selector('p')
            lst2 = []
            for e in resultados:
                lst2.append(e.text)
            lst2
            text_ = []
            for i in lst2:
                if i is not '':
                    text_.append(i)
            texto = '-'.join(text_)


            titular = driver.find_element_by_class_name('titular').text

            fecha = driver.find_element_by_css_selector('html body.espana span#cataluna.main main.contenido-articulo span.cuerpo-texto span.data span.datos-articulo span.datos span.fecha time.actualizado').text

            autor = driver.find_element_by_class_name('nombre').text

            abc_dict = {'newspaper': 'abc.com', 'link': link,'title': titular, 'author': autor, 'date': str(fecha), 'text': texto}

            df_abc_ = pd.DataFrame(abc_dict, index=[0])
            
            df_abc_['date'] = df_abc_['date'].replace('\ \d.+', '')
        
            df_list.append(df_abc_)
            driver.close()
            
            df_final_abc = pd.concat(df_list)
            
        except:
            continue
            
    df_ = df_final_abc.merge(df_abc, how='left')
        
    df_ = df_[['keyword', 'newspaper','link', 'title', 'author', 'date', 'text']]

    return df_

In [44]:
def save_function(df_to_save):
    today = str(date.today())
    nombre_archivo = 'noticias_abc_' + today + '.csv'
    path = '../DFGM/DATA_news'
    archivo_de_salida = os.path.join(path,nombre_archivo)
    df_to_save.to_csv(archivo_de_salida, index=False)

In [45]:
def abc_pipeline(lista_keywords):
    starttime = datetime.datetime.now()
    url_extractor(lista_keywords)
    df_abc = concatenator()
    
    df_to_save = scrap_news(df_abc)
    save_function(df_to_save)

    endtime = datetime.datetime.now()
    duration = endtime - starttime
    
    print(duration)
    return df_to_save

In [46]:
abc_pipeline(lista_keywords)

Negro de mierda
https://www.abc.es/hemeroteca/Negro de mierda/pagina-1
Búsqueda avanzada
Buscando negro%20de%20mierda
461 resultados encontrados.Página 1 de 47
Buscando negro%20de%20mierda
https://www.abc.es/hemeroteca/Negro de mierda/pagina-2
Búsqueda avanzada
Buscando negro%20de%20mierda
461 resultados encontrados.Página 2 de 47
Buscando negro%20de%20mierda
https://www.abc.es/hemeroteca/Negro de mierda/pagina-3
Búsqueda avanzada
Buscando negro%20de%20mierda
461 resultados encontrados.Página 3 de 47
Buscando negro%20de%20mierda
https://www.abc.es/hemeroteca/Negro de mierda/pagina-4
Búsqueda avanzada
Buscando negro%20de%20mierda
461 resultados encontrados.Página 4 de 47
Buscando negro%20de%20mierda
https://www.abc.es/hemeroteca/Negro de mierda/pagina-5
Búsqueda avanzada
Buscando negro%20de%20mierda
461 resultados encontrados.Página 5 de 47
Buscando negro%20de%20mierda
https://www.abc.es/hemeroteca/Negro de mierda/pagina-6
Mantero


KeyboardInterrupt: 